In [70]:
# import libraries
from pyvis.network import Network  # Interactive network visualization
import pandas as pd # Data Frame
import networkx as nx # Network analysis
from tqdm.notebook import tqdm # Progress bar
import webbrowser


In [71]:
# Read data
ratings_df = pd.read_csv('data/BX-Book-Ratings.csv', sep=';', encoding='latin-1')
users_df = pd.read_csv('data/BX-Users.csv', sep=';', encoding='latin-1')
books_df = pd.read_csv('data/BX-Books.csv', sep=';', encoding='latin-1', on_bad_lines='skip')

/tmp/ipykernel_189408/2220143568.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv('data/BX-Books.csv', sep=';', encoding='latin-1', on_bad_lines='skip')


In [72]:
books_df.columns = books_df.columns.str.lower()
books_df.columns = books_df.columns.str.replace('-','_')
books_df

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [73]:
users_df.columns = users_df.columns.str.lower()
users_df.columns = users_df.columns.str.replace('-','_')
users_df

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [74]:
ratings_df.columns = ratings_df.columns.str.lower()
ratings_df.columns = ratings_df.columns.str.replace('-','_')
ratings_df

,user_id,isbn,book_rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [75]:
user_counts = ratings_df['user_id'].value_counts()

# Identify user_ids with more than 5000 occurrences
high_frequency_users = user_counts[user_counts > 100].index

# Filter out these user_ids from df
filtered_df = ratings_df[~ratings_df['user_id'].isin(high_frequency_users)]

In [76]:
df = filtered_df

In [77]:
def getColor(num):
    selection = num % 3
    table = {
        0:"#611C35",
        1:"#2E5077",
        2:"#FFA630",
    }
    return table[selection]

def getSize(num):
    selection = num % 3
    table = {
        0:600,
        1:400,
        2:200,
    }
    return table[selection]

In [78]:
getSize(2)

200

In [79]:
def buildLayers(_target_user_id, _layer_count, _df, _G:nx.Graph):
    _G.add_node(f"0_{_target_user_id}", label=str(_target_user_id), color="#611C35",size = getSize(0),title=f"User ID: {_target_user_id}")
    # Building the tree
    current_layer_users = {_target_user_id}
    for layer in tqdm(range(1, _layer_count + 1), desc="Building the layer"):
        if layer % 2 == 1:  # Odd layers: find books
            books = _df[_df['user_id'].isin(current_layer_users)]['isbn'].unique()
            for book in tqdm(books, desc=f"Building the book layer {layer}"):
                try:
                    book_row = books_df[books_df['isbn'] == book].iloc[0]
                    book_title = f"title:{book_row['book_title']};\nisbn:{book_row['isbn']}"
                    _G.add_node(f"{layer}_{book}", color=getColor(layer),size = getSize(layer), title=book_title)
                    for user in current_layer_users:
                    # Check if the user has rated this book
                        if not _df[(_df['user_id'] == user) & (_df['isbn'] == book)].empty:
                            _G.add_edge(f"{layer-1}_{user}", f"{layer}_{book}", color=getColor(layer))
                except IndexError:
                    books.discard(book)
            current_layer_users = set(books)
        else:  # Even layers: find users
            users = _df[_df['isbn'].isin(current_layer_users)]['user_id'].unique()
            for user in tqdm(users, desc=f"Building the user layer {layer}"):
                try:
                    user_title = f"{users_df[users_df['user_id'] == user].iloc[0]['user_id']}"
                    _G.add_node(f"{layer}_{user}", color=getColor(layer),size = getSize(layer), title=f"User ID: {user_title}")
                    for book in current_layer_users:
                        # Check if the user has rated this book
                        if not _df[(_df['user_id'] == user) & (_df['isbn'] == book)].empty:
                            _G.add_edge(f"{layer-1}_{book}", f"{layer}_{user}", color=getColor(layer))
                except IndexError:
                    users.discard(user)
                
            current_layer_users = set(users)

In [80]:
# Initialize Pyvis Network
net = Network(notebook=True,select_menu=True)
# net.show_buttons(filter_=['physics'])
net.barnes_hut()
G = nx.Graph()

# Initialize variables
target_user_id = 221707
layer_count = 2
initial_size= 1000


buildLayers(target_user_id,layer_count,df,G)
print(f"Number of nodes: {len(G.nodes)}")
print(f"Number of edges: {len(G.edges)}")
net.from_nx(G)
# Display the network
filename = f'user_book_network_{target_user_id}.html'
net.write_html(filename)
file = open(filename,'r')
content = file.read()
content = content.replace('id="mynetwork"', 'id="mynetwork" style="height: 100dvh;"')
file.close()
file = open(filename,'w')
file.write(content)
file.close()
webbrowser.open(f'{filename}')

Building the layer:   0%|          | 0/2 [00:00<?, ?it/s]

Building the book layer 1:   0%|          | 0/7 [00:00<?, ?it/s]

Building the user layer 2:   0%|          | 0/289 [00:00<?, ?it/s]

Number of nodes: 297
Number of edges: 308


True